In [1]:
import statistics

In [10]:
aifile = '../../../../../figshare/alien_index/alien_index_recipientPLACOBRANCHOIDEA_ancestralMETAZOA.txt'
gfffile = '../../../../../figshare/annotation/genes/Elysia_crispata_ECLA1_v1.gff'
outfile = '../../../../../figshare/alien_index/Elysia-flye-racon1-medaka-nextpolish-contig-ai-summary.txt'
skiplist = []

In [11]:
aidict = {}

fi = open(aifile)

for line in fi:
    #print(line)
    if line[0] == '#':
        continue
        
    ai = float(line.split('\t')[13])
    gene = line.split('\t')[0].split('.')[0]
    #print(gene,ai)
    
    aidict[gene] = ai
    
fi.close()

In [12]:
#!grep gene {gfffile}

In [13]:
contigdict = {}
genedict = {}
contiglookup = {}

fi = open(gfffile)

for line in fi:
    #print(line)
    col = line.rstrip().split('\t')


    if col[2] == 'gene':
        contig = col[0]
        if contig not in contigdict:
            contigdict[contig] = []
            genedict[contig] = []


        gene = col[8].split('ID=')[1].split(';')[0]
        contiglookup[gene] = contig

        #print(contig,gene)
        
        if gene in aidict:
            ai = aidict[gene]
            #print(contig, gene, ai)
            
            contigdict[contig].append(ai)
            genedict[contig].append(gene)

fi.close()

In [16]:
fo = open(outfile, 'w')
fo.write("#ai genes are those with with ai greater than 0.1\n")
fo.write("#contigs were excluded if percent ai genes greater than or equal to 0.5 and/or minimum ai for all genes on a contig was greater than zero\n")
fo.write(f'#contig\tnum genes\tnum ai genes\tpercent ai genes\tavg ai\tmedian ai\tmin ai\tmax ai\tgenelist\n')

for contig in contigdict:
    numgenes = len(contigdict[contig])
    if numgenes > 0:
        avgai = sum(contigdict[contig]) / len(contigdict[contig])
        medai = statistics.median(contigdict[contig])
        minai = min(contigdict[contig])
        maxai = max(contigdict[contig])
        countai = len([i for i in contigdict[contig] if i > 0.1])
        perai = countai / numgenes
        genelist = ', '.join(genedict[contig])
        #print(contig,numgenes,avgai,medai,minai,maxai, genelist)
        fo.write(f'{contig}\t{numgenes}\t{countai}\t{perai}\t{avgai}\t{medai}\t{minai}\t{maxai}\t{genelist}\n')
        
        if perai >= 0.5 or minai > 0:
            skiplist.append(contig)
fo.close()

In [19]:
fo = open('../../../02_Assembly/Elysia-flye-racon1-medaka-nextpolish-contigs-to-exclude.txt', 'w')

for contig in skiplist:
    fo.write(contig + '\n')
fo.close()

In [15]:
fi = open(aifile)
fo = open('../../../11_Other_analyses/alien_index/alien_index_recipientPLACOBRANCHOIDEA_ancestralMETAZOA_filtered.txt', 'w')
figo = open('../figures/ai_table.Rin', 'w')
figo.write('gene\tnormbitscoremetazoa\tnormbitscoreother\tai\tnumhits\n')

for line in fi:
    #print(line)
    if line[0] == '#':
        fo.write(line)
        continue
        
    ai = float(line.split('\t')[13])
    gene = line.split('\t')[0].split('.')[0]
    #print(gene,ai)
    subclade = line.split('\t')[2]
    contig = contiglookup[gene]
    nbmeta = line.split('\t')[8]
    nbother = line.split('\t')[12]
    numhits = line.rstrip().split('\t')[19]
    if numhits == '>200':
        numhits = 200
    #print(gene,nbmeta,nbother,numhits)
    
    numhits = int(numhits)
    if numhits > 50:
        if contig not in skiplist:
            figo.write(gene + '\t' + nbmeta + '\t' + nbother + '\t' + str(ai) + '\t' + str(numhits) + '\n')
    
            #print(gene,ai,contig)
            if ai > 0.1:
                fo.write(line)
                #print(gene,ai,contig)
            
fi.close()
fo.close()
figo.close()